# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [20]:
import numpy as np

X = np.array((
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [0,1,0],
    [1,0,0],
    [1,1,1],
    [0,0,0]
             ), dtype=float)

y = np.array((
    [0],
    [1],
    [1],
    [1],
    [1],
    [0],
    [0]
        ), dtype=float)


In [27]:
X.shape

(7, 3)

In [21]:
class NeuralNetwork:
    def __init__(self):
        # 3 input, 4 hidden, 1 output layers
        self.input = 3
        self.hiddenNodes = 4
        self.outputNodes = 1
        
        # initial weights
        self.weights1 = np.random.randn(self.input, self.hiddenNodes)
        
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
    
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        # weighted sums of inputs and hidden
        self.hidden_sum = np.dot(X, self.weights1)
        
        #activations of weighted sums
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) # adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) # adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)        

In [22]:
nn = NeuralNetwork()

nn.train(X, y)
nn.feed_forward(X).shape

(7, 1)

In [3]:
nn = NeuralNetwork()

for epoch in range(1000):
    nn.train(X, y)
print('+' + '---' * 3 + f'EPOCH {epoch+1}' + '---'*3 + '+')
print('Input: \n', X)
print('Actual Output: \n', y)
print('Weights1: ', nn.weights1)
print('Weights2:', nn.weights2)
print('Predicted Output: \n', str(nn.feed_forward(X)))
print('Loss: \n', str(np.mean(np.square(y - nn.feed_forward(X)))))

    

+---------EPOCH 1000---------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Weights1:  [[ 1.36920918  3.67880761  3.41247584 -5.99568638]
 [ 1.44623696  3.67956849  2.7875351  -6.03396476]
 [-5.92391277 -5.15983317  2.70126155  1.17453544]]
Weights2: [[  7.9284813 ]
 [ -7.63694166]
 [  4.23034784]
 [-10.06571759]]
Predicted Output: 
 [[0.02302148]
 [0.94314627]
 [0.94311511]
 [0.94928113]
 [0.9499095 ]
 [0.09066361]
 [0.05886084]]
Loss: 
 0.0033948824076551463


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [36]:
from keras.datasets import mnist

In [37]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [23]:
class NeuralNetwork:
    def __init__(self):
        # 28 input, 4 hidden, 9 output layers
        self.input = 28
        self.hiddenNodes = 29
        self.outputNodes = 9
        
        # initial weights
        self.weights1 = np.random.randn(self.input, self.hiddenNodes)
        
        self.weights2 = np.random.randn(self.hiddenNodes, self.outputNodes)
    
    def sigmoid(self, s):
        return 1 / (1 + np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def feed_forward(self, X):
        # weighted sums of inputs and hidden
        self.hidden_sum = np.dot(X, self.weights1)
        
        #activations of weighted sums
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, X, y, o):
        
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) # adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) # adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [31]:
x_train.shape

(60000, 28, 28)

In [26]:
nn = NeuralNetwork()
nn.feed_forward(x_train).shape

(60000, 28, 9)

In [7]:
for epoch in range(1000):
    nn.train(x_train, y_train)
print('+' + '---' * 3 + f'EPOCH {epoch+1}' + '---'*3 + '+')
print('Input: \n', X)
print('Actual Output: \n', y)
print('Weights1: ', nn.weights1)
print('Weights2:', nn.weights2)
print('Predicted Output: \n', str(nn.feed_forward(X)))
print('Loss: \n', str(np.mean(np.square(y - nn.feed_forward(X)))))

ValueError: operands could not be broadcast together with shapes (60000,) (60000,28,9) 

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?

In [38]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(100, ), 
                    max_iter=480, alpha=1e-4,
                    solver='sgd', verbose=10, 
                    tol=1e-4, random_state=1,
                    learning_rate_init=.1)
train_labels = y_train.reshape(y_train.shape[0],)
print(x_train.shape, train_labels.shape)
mlp.fit(x_train, train_labels)
print("Training set score: %f" % mlp.score(x_train, train_labels))
print("Test set score: %f" % mlp.score(x_test, y_test))
help(mlp.fit)

(60000, 28, 28) (60000,)


ValueError: Found array with dim 3. Estimator expected <= 2.